In [ ]:
--%%sql

SELECT "year",
	COUNT("year")
	FROM "emails-text"
	WHERE "year" = 2020
	GROUP BY "year";

In [ ]:
--%%sql

CREATE INDEX "emails-text-year" ON "emails-text" ("year");

In [ ]:
--%%sql
 
SELECT *
FROM "emails-text" AS E
JOIN "travel-disclosures" AS T ON T."Year" = E."year"
ORDER BY E."year";

In [ ]:
--%%sql

CREATE INDEX "travel-disclosures-Year" ON "travel-disclosures" ("Year");

In [ ]:
--%%sql

SELECT *
FROM "emails-text" AS E
JOIN "travel-disclosures" AS T ON T."FilerName" = E."name"
WHERE T."MemberName" = 'McCarthy, Kevin'
ORDER BY E."year" DESC
LIMIT 100;

In [ ]:
--%%sql

CREATE INDEX "travel-disclosures-MemberName-hash" ON "travel-disclosures" USING HASH ("MemberName");
CREATE INDEX "emails-text-name-hash" ON "emails-text" USING HASH ("name");

In [ ]:
--%%sql

CREATE FUNCTION FIRSTLAST(text) RETURNS text AS $$
	SELECT REGEXP_REPLACE(REGEXP_REPLACE($1,'[^A-Za-z\.,\s]', ''), '^([^\s]+), ([^\s]+)\s*.*',  '\2 \1');
$$ LANGUAGE SQL;

In [ ]:
--%%sql

SELECT T."MemberName",
	E."name",
  E.date::timestamp with time zone - T."ReturnDate"::timestamp with time zone AS "since-return",
	T."TravelSponsor",
	E."subject",
	E."body"
FROM "travel-disclosures" AS T
JOIN "emails-text" AS E ON E.NAME = FIRSTLAST(T."MemberName")
WHERE
	(E."date"::TIMESTAMPTZ - T."ReturnDate"::TIMESTAMPTZ) < '1 week'::interval
	AND (E."date"::TIMESTAMPTZ - T."ReturnDate"::TIMESTAMPTZ) > '0 days'::interval;

In [ ]:
--%%sql

CREATE MATERIALIZED VIEW IF NOT EXISTS "email-travel-return-diff"
AS
 SELECT t."MemberName" AS "travel-MemberName",
    e.name AS "email-name",
    e.date::timestamp with time zone - t."ReturnDate"::timestamp with time zone AS "since-return",
    t."TravelSponsor" AS "travel-TravelSponsor",
    e.subject AS "email-subject",
    e.body AS "email-body"
   FROM "travel-disclosures" t
     JOIN "emails-text" e ON e.name::text = firstlast(t."MemberName"::text)
WITH DATA;

CREATE INDEX "email-travel-return-diff-since-return" ON "email-travel-return-diff" ("since-return");

In [ ]:
--%%sql

SELECT "travel-MemberName", *
FROM "email-travel-return-diff" AS ET
WHERE "since-return" > '0 days'::interval
	AND "since-return" < '1 week'::interval;

In [ ]:
--%%sql

SELECT *
FROM "email-travel-return-diff" AS ET
WHERE "since-return" > '0 days'::interval
	AND "since-return" < '1 week'::interval
	AND TO_TSVECTOR("email-body") @@ TO_TSQUERY('climate');

In [ ]:
--%%sql

ALTER TABLE "emails-text"
    ADD COLUMN "search-index-column" tsvector
               GENERATED ALWAYS AS (to_tsvector('english', coalesce("subject", '') || ' ' || coalesce("body", ''))) STORED;
CREATE INDEX "search-index-column-gin" ON "emails-text" USING GIN ("search-index-column");

In [ ]:
--%%sql

SELECT *
FROM "emails-text"
WHERE "search-index-column" @@ TO_TSQUERY('climate & fraud');

In [ ]:
--%%sql

SELECT *
FROM "stocks"
WHERE "jdoc" @> '{"cap_gains_over_200_usd": true}'::JSONB;

In [ ]:
--%%sql

CREATE INDEX "stocks-index-gin" ON "stocks" USING GIN ("jdoc");

In [ ]:
--%%sql

ALTER TABLE "stocks"
	ADD COLUMN "min-amount" int
		GENERATED ALWAYS AS (

	(REGEXP_REPLACE((REGEXP_REPLACE(
		("jdoc"->'amount')::text,
		'\$([0-9,]+)\s*[-+]*\s*\$*([0-9,]*)',
		'\1'
	))::text || '-1', '[^0-9]', '', 'g'))::int
			
) STORED;
CREATE INDEX "stocks-min-amount" ON "stocks" ("min-amount");

In [ ]:
--%%sql

SELECT *
FROM "stocks"
WHERE "min-amount" > 5000001
ORDER BY "min-amount" DESC;


SELECT "jdoc"->'comment'
FROM "stocks"
WHERE "jdoc" ? 'comment' and ("jdoc"->'comment')::text like '%Child%';


SELECT "jdoc"->'comment'
FROM "stocks"
WHERE "jdoc" ? 'comment' and ("jdoc"->'comment')::text != '"--"';
